<a href="https://colab.research.google.com/github/tanvircr7/meh/blob/master/DL_App_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

def segment_script(text, min_segment_size=3, parse_specificity=0.09):
    """
    Segments a script into coherent sections using topic similarity

    Args:
        text (str): The input script text
        min_segment_size (int): Minimum number of sentences per segment

    Returns:
        list: List of text segments
    """
    # Download required NLTK data
    nltk.download('punkt')
    nltk.download('punkt_tab')

    # Split into sentences
    sentences = sent_tokenize(text)

    # Create TF-IDF vectors for each sentence
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Calculate similarity between adjacent sentences
    similarity_scores = []
    for i in range(len(sentences)-1):
        similarity = (tfidf_matrix[i] * tfidf_matrix[i+1].T).toarray()[0][0]
        similarity_scores.append(similarity)

    # Find segment boundaries where similarity is low
    threshold = np.mean(similarity_scores) - parse_specificity*np.std(similarity_scores)
    boundaries = [0]
    current_size = 0

    for i, score in enumerate(similarity_scores):
        current_size += 1
        if score < threshold and current_size >= min_segment_size:
            boundaries.append(i + 1)
            current_size = 0

    boundaries.append(len(sentences))

    # Create segments
    segments = []
    for i in range(len(boundaries)-1):
        segment = ' '.join(sentences[boundaries[i]:boundaries[i+1]])
        segments.append(segment)

    return segments

# Example usage
sample_text = """
Artificial intelligence has revolutionized many fields. From healthcare to transportation,
AI systems are making our lives easier. Machine learning algorithms can now recognize patterns
that humans might miss. This has led to breakthrough discoveries in medicine.

The ethics of AI is another important consideration. We must ensure AI systems are fair
and unbiased. Privacy concerns have also emerged as AI systems collect more data.
These challenges require careful thought and regulation.

Looking to the future, AI will continue to evolve. New architectures and algorithms
are being developed. The potential applications seem limitless. However, we must
proceed with caution and wisdom.
"""

segments = segment_script(sample_text)

for i, segment in enumerate(segments):
    print(f"\nSegment {i+1}:")
    print(segment)
    print(type(segment))


Segment 1:

Artificial intelligence has revolutionized many fields. From healthcare to transportation,
AI systems are making our lives easier. Machine learning algorithms can now recognize patterns
that humans might miss.
<class 'str'>

Segment 2:
This has led to breakthrough discoveries in medicine. The ethics of AI is another important consideration. We must ensure AI systems are fair
and unbiased. Privacy concerns have also emerged as AI systems collect more data.
<class 'str'>

Segment 3:
These challenges require careful thought and regulation. Looking to the future, AI will continue to evolve. New architectures and algorithms
are being developed.
<class 'str'>

Segment 4:
The potential applications seem limitless. However, we must
proceed with caution and wisdom.
<class 'str'>


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [10]:
len(segments)

4

In [19]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
import numpy as np

def extract_keywords(text, num_keywords=5):
        """
        Extract key phrases from text segment for better prompt generation
        """
        # Tokenize and remove stopwords
        # lemmatizer = WordNetLemmatizer()
        # lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        # text = lemmatized_words
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        tokens = [t for t in tokens if t.isalnum() and t not in stop_words]

        # Get word frequencies
        freq_dist = nltk.FreqDist(tokens)

        # Get most common words
        keywords = [word for word, freq in freq_dist.most_common(num_keywords)]

        return keywords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
for segment in segments:
  print(extract_keywords(text=segment))

['artificial', 'intelligence', 'revolutionized', 'many', 'fields']
['ai', 'systems', 'led', 'breakthrough', 'discoveries']
['challenges', 'require', 'careful', 'thought', 'regulation']
['potential', 'applications', 'seem', 'limitless', 'however']


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np


